In [ ]:
#Adding dependecies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from config import g_key
import pycountry
import requests
import json
import gmaps

In [ ]:
#importing data set
covid_data=pd.read_csv("data/country_vaccinations.csv")
covid_data.head()

In [ ]:
# % of population in the data set is presented up to date
#finding last value for each country, to find the most update value
population_vacc=covid_data[["country","people_fully_vaccinated_per_hundred"]].drop_duplicates(subset=['country'], keep="last")
population_vacc.head()

In [ ]:
#calulacting the sum of all people fully vaccinated in each country
selected=covid_data[["country","people_fully_vaccinated"]].groupby(["country",]).sum()
selected.head()

In [ ]:
country_name=covid_data["country"].unique()
iso_code=covid_data["iso_code"].unique()
#Creating new data frame
countries_summary=pd.DataFrame({"Country":country_name,
                                "ISO code": iso_code,
                              "%  of population vaccinated":population_vacc["people_fully_vaccinated_per_hundred"]})
countries_summary=countries_summary.set_index("Country")
countries_summary["Number of people fully vaccinated"]=selected["people_fully_vaccinated"]
countries_summary.head()
countries_summary.to_csv("Worldwide_vaccination_summary.csv", index=True, header=True)

In [ ]:
#converting countries iso_code3 into iso_code alpha2
countries = {}
for country in pycountry.countries:
    countries[country.alpha_3] = country.alpha_2

iso_code2 = [countries.get(country, 'Unknown code') for country in iso_code]

countries_summary["ISO code"]=iso_code2
countries_summary.head()

In [ ]:
base_url="https://maps.googleapis.com/maps/api/geocode/json?"

In [ ]:
#creating google maps API call to find latitude and longitude for each country
latitude=[]
longtitude=[]
for code in iso_code2:
    try:
        target_url=f"{base_url}components=country:{code}&key={g_key}"
        location = requests.get(target_url).json()
        latitude.append(location["results"][0]["geometry"]["location"]["lat"])
        longtitude.append(location["results"][0]["geometry"]["location"]["lng"])
#if country not found appending pd.NaT to make sure the latitude and longtitude lenghts are matching the lenght of data frame
    except:
        latitude.append(pd.NaT)
        longtitude.append(pd.NaT)
        print("Country code not found. Processing further...")
print("Data retrieval completed")

In [ ]:
#Adding locations to dataframe
countries_summary["Latitude"]=latitude
countries_summary["Longtitude"]=longtitude

In [ ]:
#Plotting the heatmap of Number of people fully vaccinated
#Configure gmaps
gmaps.configure(api_key=g_key)
#Using the Lat and Lng as locations and Number of people fully vaccinated as the weight

# Fill NaN values and convert to float
countries=countries_summary.dropna(subset=["Latitude"])
countries.head()

In [ ]:
loc=countries[["Latitude","Longtitude"]]

In [ ]:
people_vacc=countries["Number of people fully vaccinated"].astype(float)

In [ ]:
# Plot Heatmap
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(loc,weights=people_vacc)
fig.add_layer(heatmap_layer)
fig